# Ein Custom LLM Model mit Langchain und GPT4All

In [ ]:
!wget -O /content/drive/MyDrive/Forschungstudie_Beckmann/model.gguf "https://gpt4all.io/models/gguf/wizardlm-13b-v1.2.Q4_0.gguf".   # /content/drive/MyDrive/Forschungstudie_Beckmann/model.gguf(Eigener Pfad eingeben)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

## Load Data

In [ ]:
loader = PyPDFLoader("/content/drive/MyDrive/Forschungstudie_Beckmann/Hochschule_Heilbronn_Wikipedia.pdf")
documents = loader.load_and_split()
# print(documents[0].page_content)

## Chunking

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(documents)
print(texts[0].page_content)

## Embedding

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="deutsche-telekom/gbert-large-paraphrase-cosine")

In [ ]:
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

## Creating Chain

In [ ]:
model_n_ctx = 1000
model_path = "/content/drive/MyDrive/Forschungstudie_Beckmann/model.gguf"
callbacks = [StreamingStdOutCallbackHandler()]
llm= GPT4All(model=model_path, callbacks=callbacks, verbose=True,n_threads=16, temp=0.5)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    verbose=False,
)



## Ask Questions

In [ ]:
%%time
res = qa(
    "Wann ist HHN gegründet?"
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 Die Gründung von HHN (Heilbronner Institut für Lebenslanges Lernen) erfolgte am 17. April 1961, als die Vorlaufschule "Staatliche Ingenieurschule Heilbronn" gegründet wurde.CPU times: user 6h 42min 38s, sys: 39min 58s, total: 7h 22min 37s
Wall time: 6h 13min 3s


In [ ]:
print(res["result"])

In [ ]:
%%time
prompt = f"""How much is the investment amount in Microsoft on 6/22? Extract the answer from the text."""
res = qa(prompt.strip())

In [ ]:
print(res["result"])